[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eirasf/GCED-AA2/blob/main/lab4/lab4_parte1.ipynb)
# Práctica 4: Redes neuronales usando Keras con Regularización
## Parte 1. Early Stopping
### Overfitting
El problema del sobreajuste (*overfitting*) consiste en que la solución aprendida se ajusta muy bien a los datos de entrenamiento, pero no generaliza adecuadamente ante la aparición de nuevos datos. 

# Regularización

Una vez diagnosticado el sobreajuste, es hora de probar diferentes técnicas que intenten reducir la varianza, sin incrementar demasiado el sesgo y, con ello, el modelo generaliza mejor. Las técnicas de regularización que vamos a ver en este laboratorio son:
1. *Early stopping*. Detiene el entrenamiento de la red cuando aumenta el error.
1. Penalización basada	en	la	norma	de	los	parámetros (tanto norma L1 como L2). 
1. *Dropout*. Ampliamente utilizada en aprendizaje profundo, "desactiva" algunas neuronas para evitar el sobreajuste.

En esta primera parte del Laboratorio 4 nos centraremos en **Early Stopping**






## Pre-requisitos. Instalar paquetes

Para la primera parte de este Laboratorio 4 necesitaremos TensorFlow, TensorFlow-Datasets y otros paquetes para inicializar la semilla y poder reproducir los resultados

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import numpy as np
import random

#Fijamos la semilla para poder reproducir los resultados
seed=1234
os.environ['PYTHONHASHSEED']=str(seed)
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)


Además, cargamos también APIs que vamos a emplear para que el código quede más legible

In [ ]:
#API de Keras, modelo Sequential y las capas que vamos a usar en nuestro modelo
from tensorflow import keras
from keras.models import Sequential
from keras.layers import InputLayer
from keras.layers import Dense 
#Para mostrar gráficas
from matplotlib import pyplot

#Necesario para el EarlyStopping
from keras.callbacks import EarlyStopping


## Cargamos el conjunto de datos

De nuevo, seguimos empleando el conjunto *german_credit_numeric* ya empleado en los laboratorios anteriores, aunque esta vez lo dividimos para tener un subconjunto de entrenamiento, otro de validación (que nos servirá para detener el entrenamiento) y otro de test para evaluar el rendimiento del modelo.


In [ ]:
# Cargamos el conjunto de datos
ds_train = tfds.load('german_credit_numeric', split='train[:40%]',  as_supervised=True).batch(128)
ds_val = tfds.load('german_credit_numeric', split='train[40%:50%]', as_supervised=True).batch(128)
ds_test = tfds.load('german_credit_numeric', split='train[50%:]', as_supervised=True).batch(128)

También vamos a establecer la función de pérdida, el algoritmo que vamos a emplear para el entrenamiento y la métrica que nos servirá para evaluar el rendimiento del modelo entrenado.

In [ ]:
#Indicamos la función de perdida, el algoritmo de optimización y la métrica para evaluar el rendimiento 
fn_perdida = tf.keras.losses.BinaryCrossentropy()
optimizador = tf.keras.optimizers.Adam(0.001)
metrica = tf.keras.metrics.AUC()

## Creamos un modelo *Sequential*
Creamos un modelo *Sequential* tal y como se ha hecho en el Laboratorio 3. Parte 2.

In [ ]:
tamano_entrada = 24
h0_size = 20
h1_size = 10
h2_size = 5
#TODO - define el modelo Sequential
model = ...
#TODO - incluye la capa de entrada y las 4 capas Dense del modelo
......

#Construimos el modelo y mostramos 
model.build()
print(model.summary())

Completar el método *compile*.

In [ ]:
#TODO - indicar los parametros del método compile
model.compile(loss=fn_perdida,
              optimizer=optimizador,
              metrics=[metrica])

Hacemos una llamada al método *fit* usando el conjunto de entrenamiento como entrada, indicando el número de epochs y, además,  incluyendo el argumento *validation_data* que permite usar un subconjunto de datos para validar. Las diferencias entre entrenamiento y validación se pueden apreciar en el gráfico.

**NOTA**: Observad las diferencias de resultado entre entrenamiento, validación y test.

In [ ]:
#Establecemos el número de epochs
num_epochs =  700

# Guardamos los pesos antes de entrenar, para poder resetear el modelo posteriormente y hacer comparativas.
pesos_preentrenamiento = model.get_weights()

#TODO - entrenar el modelo usando como entradas el conjunto de entrenamiento, 
#indicando el número de epochs y el conjunto de validación
history = model.fit(....)

# plot training history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

In [ ]:
#TODO - llamar a evaluate usando el conjunto de test, guardando el resultado
result = model.evaluate(.....) 
print(model.metrics_names)
print(result)

## Usando Early Stopping en el entrenamiento

Keras nos facilita un *Callback* para realizar la parada temprana (*keras.callbacks.EarlyStopping*).  De este modo, podemos parar el entrenamiento cuando una determinada medida (especificada en el argumento *monitor*) empeore su rendimiento (el argumento *mode* nos dirá si se espera que dicha medida se minimice, *min*, o maximice, *max*). Opcionalmente, el usuario puede proporcionar el argumento *patience* para especificar cuantas *epochs* debe esperar el entrenamiento antes de detenerse.

**TO-DO**: Realizar varias veces el entrenamiento, cambiando los distintos parámetros para ver las diferencias en el aprendizaje. ¿Se para siempre en el mismo *epoch*? Comprobar el rendimiento en el conjunto de test.

In [ ]:
# simple early stopping
#TODO- indica la medida a monitorizar,  el modo y la paciencia
es = EarlyStopping(
    monitor=....
    mode=...
    patience=...
)

# Antes de entrenar, olvidamos el entrenamiento anterior restaurando los pesos iniciales
model.set_weights(pesos_preentrenamiento)

#TODO - entrenar el modelo usando como entradas el conjunto de entrenamiento, 
#indicando el número de epochs, el conjunto de validación y la callback para el EarlyStopping
history = model.fit(...., callbacks=[es])

# plot training history
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='val')
pyplot.legend()
pyplot.show()

Evaluación sobre el conjunto de test (no usado para el entrenamiento).

In [ ]:
#TODO - llamar a evaluate usando el conjunto de test, guardando el resultado
result = model.evaluate(....) 
print(model.metrics_names)
print(result)